Week 1

In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd 
import numpy as np

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
df = pd.read_csv('afa2e701598d20110228.csv', sep=';')
print("Original data loaded. Shape:", df.shape)
df

Original data loaded. Shape: (2861, 11)


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2861 non-null   int64  
 1   date       2861 non-null   object 
 2   NH4        2858 non-null   float64
 3   BSK5       2860 non-null   float64
 4   Suspended  2845 non-null   float64
 5   O2         2858 non-null   float64
 6   NO3        2860 non-null   float64
 7   NO2        2858 non-null   float64
 8   SO4        2812 non-null   float64
 9   PO4        2833 non-null   float64
 10  CL         2812 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 246.0+ KB


In [5]:
df['date'] = pd.to_datetime(df['date'])

C:\Users\Hp\AppData\Local\Temp\ipykernel_18072\3532345252.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'])


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         2861 non-null   int64         
 1   date       2861 non-null   datetime64[ns]
 2   NH4        2858 non-null   float64       
 3   BSK5       2860 non-null   float64       
 4   Suspended  2845 non-null   float64       
 5   O2         2858 non-null   float64       
 6   NO3        2860 non-null   float64       
 7   NO2        2858 non-null   float64       
 8   SO4        2812 non-null   float64       
 9   PO4        2833 non-null   float64       
 10  CL         2812 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 246.0 KB


In [7]:
df = df.sort_values(by=['id', 'date'])

In [8]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [9]:
pollutants = ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']

In [10]:
for station_id in df['id'].unique():
    station_data = df[df['id'] == station_id].copy()

    for p in pollutants:
        # 1. Lag Features: The value of the pollutant at previous time steps
        # We use .shift() which looks at the previous row for that station
        df.loc[station_data.index, f'{p}_lag_1'] = station_data[p].shift(1)
        df.loc[station_data.index, f'{p}_lag_3'] = station_data[p].shift(3)
        df.loc[station_data.index, f'{p}_lag_6'] = station_data[p].shift(6)

        # 2. Rolling Window Features: The trend over a recent period
        # .rolling() creates a moving window to calculate stats
        df.loc[station_data.index, f'{p}_rolling_mean_3'] = station_data[p].shift(1).rolling(window=3).mean()
        df.loc[station_data.index, f'{p}_rolling_std_3'] = station_data[p].shift(1).rolling(window=3).std()

print("Feature engineering complete.")

Feature engineering complete.


In [11]:
df = df.dropna(subset=pollutants)
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,...,PO4_lag_1,PO4_lag_3,PO4_lag_6,PO4_rolling_mean_3,PO4_rolling_std_3,CL_lag_1,CL_lag_3,CL_lag_6,CL_rolling_mean_3,CL_rolling_std_3
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,...,0.454,NaN,NaN,NaN,NaN,289.5,NaN,NaN,NaN,NaN
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,...,0.090,NaN,NaN,NaN,NaN,1792.0,NaN,NaN,NaN,NaN
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,...,0.200,0.454,NaN,0.248000,0.186687,2509.0,289.5,NaN,1530.166667,1132.679382
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,...,0.377,0.090,NaN,0.222333,0.144798,1264.0,1792.0,NaN,1855.000000,624.886390


In [12]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
id,2780.0,12.279856,1.0,8.0,13.0,16.0,22.0,6.072261
date,2780,2010-04-11 19:10:57.841726464,2000-01-10 00:00:00,2005-03-17 00:00:00,2010-05-12 00:00:00,2015-06-09 00:00:00,2021-04-07 00:00:00,NaN
NH4,2778.0,0.634145,0.0,0.08,0.22,0.48,30.26,1.829515
BSK5,2780.0,4.248248,0.0,2.12,3.745,5.7,27.12,2.760254
Suspended,2778.0,12.881451,0.0,6.0,10.0,14.4,595.0,16.655984
O2,2780.0,9.549462,0.0,7.1,9.0,11.6325,90.0,4.453361
NO3,2780.0,4.349107,0.0,1.4,2.825,5.6,133.4,6.953503
NO2,2780.0,0.224384,0.0,0.03,0.0575,0.12,109.0,2.157737
SO4,2780.0,59.671929,0.003,27.1,37.8,65.145,3573.4,97.074655
PO4,2780.0,0.393275,0.0,0.12975,0.27,0.46,11.66,0.640663


In [13]:
print(f"Shape after dropping rows with missing values: {df.shape}")

Shape after dropping rows with missing values: (2780, 43)


Week 2

In [14]:
feature_cols = [col for col in df.columns if col not in pollutants and col not in ['date']]
X = df[feature_cols]
y = df[pollutants]
print(feature_cols)

['id', 'NH4', 'BSK5', 'Suspended', 'year', 'month', 'O2_lag_1', 'O2_lag_3', 'O2_lag_6', 'O2_rolling_mean_3', 'O2_rolling_std_3', 'NO3_lag_1', 'NO3_lag_3', 'NO3_lag_6', 'NO3_rolling_mean_3', 'NO3_rolling_std_3', 'NO2_lag_1', 'NO2_lag_3', 'NO2_lag_6', 'NO2_rolling_mean_3', 'NO2_rolling_std_3', 'SO4_lag_1', 'SO4_lag_3', 'SO4_lag_6', 'SO4_rolling_mean_3', 'SO4_rolling_std_3', 'PO4_lag_1', 'PO4_lag_3', 'PO4_lag_6', 'PO4_rolling_mean_3', 'PO4_rolling_std_3', 'CL_lag_1', 'CL_lag_3', 'CL_lag_6', 'CL_rolling_mean_3', 'CL_rolling_std_3']


In [15]:
X_encoded = pd.get_dummies(X, columns=['id'], drop_first=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [17]:
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [18]:
y_pred = model.predict(X_test) # predict the output

In [19]:
print("Model Performance on the Test Data:")
for i, pollutant in enumerate(pollutants):
    print(f'Pollutant: {pollutant}')
    print('MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
    print('R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))
    print()

Model Performance on the Test Data:
Pollutant: O2
MSE: 16.462452800992804
R2: 0.24665190867551123

Pollutant: NO3
MSE: 11.385665408577696
R2: 0.6965616754995773

Pollutant: NO2
MSE: 0.48271269406672684
R2: -2.6142255487061687

Pollutant: SO4
MSE: 689.7611448460799
R2: 0.8318116791302386

Pollutant: PO4
MSE: 0.23224448535107922
R2: 0.591047666121036

Pollutant: CL
MSE: 11531.818099195929
R2: 0.9126561105480842



In [23]:
station_id = '14'
year_input = 2025
month_input = 7
input_data = pd.DataFrame({'year': [year_input], 'month':[month_input], 'id': [station_id]})
input_encoded = pd.get_dummies(input_data, columns=['id'])

# Align with training feature columns
missing_cols = set(X_encoded.columns) - set(input_encoded.columns)
for col in missing_cols:
    input_encoded[col] = 0
input_encoded = input_encoded[X_encoded.columns]  # reorder columns

# Predict pollutants
predicted_pollutants = model.predict(input_encoded)[0]

print(f"\nPredicted pollutant levels for station '{station_id}' in {year_input}-{month_input}:")
for p, val in zip(pollutants, predicted_pollutants):
    print(f"  {p}: {val:.2f}")


Predicted pollutant levels for station '14' in 2025-7:
  O2: 5.58
  NO3: 12.19
  NO2: 6.64
  SO4: 46.71
  PO4: 0.57
  CL: 58.69


In [21]:
import joblib

joblib.dump(model, 'prediction_model.pkl')
joblib.dump(X_encoded.columns.tolist(), "model_columns.pkl")
print('Model and columns structure are saved!')

Model and columns structure are saved!
